In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from model_details import MODEL_DETAILS_DICT, load_model, ModelDetails


In [45]:
import os
import json
from glob import glob
from collections import namedtuple

import torch
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from bbb.utils.pytorch_setup import DEVICE
from bbb.utils.plotting import plot_weight_samples
from bbb.config.constants import KL_REWEIGHTING_TYPES, PRIOR_TYPES, VP_VARIANCE_TYPES
from bbb.config.parameters import Parameters, PriorParameters
from bbb.models.dnn import ClassificationDNN
from bbb.models.bnn import ClassificationBNN
from bbb.data import load_mnist, load_adverserial_mnist
from bbb.models.layers import BaseBFC


from labellines import labelLines

from evaluate_classifier import evaluate_classifier
from model_details import MODEL_DETAILS_DICT, load_model, ModelDetails

import pickle

In [33]:
# Running 50 inference samples across MoG 600 1200 
net, params = load_model("bnn_mog_800")
net.inference_samples = 50
X_val = load_mnist(train=False, batch_size=params.batch_size, shuffle=True)
res = evaluate_classifier(net, X_val)

print(1 - res['eval_score'])

2022-03-29 20:44:39,235 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-29 20:44:39,236 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-29 20:44:39,241 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-29 20:44:39,241 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-29 20:44:39,242 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-29 20:44:39,243 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5


tensor(0.0186)


In [34]:
res['eval_score'].item()

0.981370210647583

In [21]:
(1 - ___['eval_score'])

tensor(0.0137)

In [2]:
matplotlib.style.use('default')
plt.rcParams.update({'axes.titlesize': 'large', 'axes.labelsize': 'medium'})
colors_hex = {'blue': '#1F77B4', 'orange': '#FF7F0E', 'green': '#2CA02C'}
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [35]:
mog_directory = os.fsencode("../saved_models/BBB_classification/baseline/mog_prior/")
sgp_directory = os.fsencode("../saved_models/BBB_classification/baseline/single_gaussian_prior/")
X_val = load_mnist(train=False, batch_size=params.batch_size, shuffle=True)

eval_dict = {}
dirs = [sgp_directory, mog_directory]
for directory in dirs:
    for root, subdirs, files in os.walk(directory):
        root = os.fsdecode(root)

        if root.split('/')[-1].__contains__(".") and root.split('/')[-2].__contains__("0"): # dst folder (time stamp, hidden units)
            
            model_name = "/".join(root.split('/')[-4:])
            
            with open(os.path.join(root, 'params.txt'), 'r') as f:
                params_dict = json.load(f)

            # Need to deserialise the prior_params into a PriorParameters object
            if params_dict['prior_params']:
                params_dict['prior_params'] = PriorParameters(**params_dict['prior_params'])

            params = Parameters(**params_dict)

            net = ClassificationBNN(params=params, eval_mode=True)
            net.model.load_state_dict(torch.load(os.path.join(root, 'model.pt'), map_location=torch.device('cpu')))

            net.inference_samples = 50
        
            res = evaluate_classifier(net, X_val)
        
            test_error = 1 - res['eval_score'].item()

            # eval_metric = np.load(os.path.join(root, 'eval_metric.npy'))
            # eval_dict[model_name] = round(1-eval_metric[-1], 4)
        
            eval_dict[model_name] = round(test_error, 5)

2022-03-29 21:19:40,355 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:19:40,355 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:19:40,365 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:19:40,366 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:19:40,367 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:19:40,367 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:23:45,006 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:23:45,006 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:23:45,008 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-29 21:23:45,009 - bbb.models.layers - INFO

In [36]:
eval_dict

{'single_gaussian_prior/300_epochs/1200/2022-03-15-09.18.07': 0.01573,
 'single_gaussian_prior/300_epochs/400/2022-03-15-14.26.34': 0.01552,
 'single_gaussian_prior/300_epochs/800/2022-03-15-14.25.46': 0.01593,
 'single_gaussian_prior/600_epochs/1200/2022-03-27-20.30.22': 0.01232,
 'mog_prior/sigma_1_exp_2_sigma_2_exp_6/1200/2022-03-27-00.06.29': 0.02324,
 'mog_prior/sigma_1_exp_2_sigma_2_exp_6/400/2022-03-27-07.52.03': 0.02113,
 'mog_prior/sigma_1_exp_2_sigma_2_exp_6/800/2022-03-26-23.55.35': 0.02033,
 'mog_prior/sigma_1_exp_1_sigma_2_exp_6/1200/2022-03-26-19.41.39': 0.01873,
 'mog_prior/sigma_1_exp_1_sigma_2_exp_6/400/2022-03-26-19.39.24': 0.01552,
 'mog_prior/sigma_1_exp_1_sigma_2_exp_6/800/2022-03-26-20.37.06': 0.01783,
 'mog_prior/sigma_1_1_sigma_2_0.2/1200/2022-03-15-18.38.27': 0.01372,
 'mog_prior/sigma_1_1_sigma_2_0.2/400/2022-03-15-21.48.39': 0.01422,
 'mog_prior/sigma_1_1_sigma_2_0.2/800/2022-03-15-18.43.25': 0.01482,
 'sigma_1_1_sigma_2_exp_7/1200/300_epochs/2022-03-27-07.54

In [47]:
another_dict = {}
eval_dict = {}
for MODEL in MODEL_DETAILS_DICT.keys():
    net, params = load_model(MODEL)
    eval_metric = np.load(os.path.join(MODEL_DETAILS_DICT[MODEL].dir, 'eval_metric.npy'))
    res = evaluate_classifier(net, X_val)

    test_error = 1 - res['eval_score'].item()

    eval_dict[MODEL] = round(1-eval_metric[-1], 5)

    another_dict[MODEL] = test_error
    
    print(MODEL, round(test_error, 5))

2022-03-30 00:47:12,284 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:47:12,284 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:47:12,294 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:47:12,295 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:47:12,296 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:47:12,296 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:03,113 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:03,114 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:03,119 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:03,119 - bbb.models.layers - INFO

bnn_sgp_1200 0.01502


2022-03-30 00:48:35,585 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:35,585 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:35,587 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:35,587 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:35,589 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:35,589 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0


bnn_sgp_800 0.01683


2022-03-30 00:48:49,760 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:49,760 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:49,770 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:49,770 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:49,771 - bbb.models.layers - INFO - Weights Prior: Gaussian with mean 0 and variance 1.0
2022-03-30 00:48:49,771 - bbb.models.layers - INFO - Biases Prior: Gaussian with mean 0 and variance 1.0


...

bnn_mog_600_400 0.01422


2022-03-30 00:53:59,788 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:53:59,789 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:53:59,794 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:53:59,794 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:53:59,796 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:53:59,796 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5


bnn_mog_1200 0.01562


2022-03-30 00:54:47,391 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:54:47,392 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:54:47,394 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:54:47,394 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:54:47,396 - bbb.models.layers - INFO - Weights Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5
2022-03-30 00:54:47,396 - bbb.models.layers - INFO - Biases Prior: Gaussian mixture with means (0, 0), variances (1.0, 0.0009118819655545162) and weight 0.5


bnn_mog_800 0.01793
bnn_mog_400 0.01472
dnn_1200 0.01693
dnn_800 0.01703
dnn_400 0.01953
dnn_600_1200 0.01883
dnn_do_400 0.01272
dnn_do_800 0.01462
dnn_do_1200 0.01532
dnn_do_600_1200 0.01372


In [50]:
for name, ter in another_dict.items():
    print(name, round(ter, 7))

bnn_sgp_1200 0.0150241
bnn_sgp_800 0.0168269
bnn_sgp_400 0.0164263
bnn_sgp_600_1200 0.0135216
bnn_sgp_600_800 0.0119191
bnn_sgp_600_400 0.013121
bnn_mog_600_1200 0.0145232
bnn_mog_600_800 0.0142227
bnn_mog_600_400 0.0142227
bnn_mog_1200 0.015625
bnn_mog_800 0.0179287
bnn_mog_400 0.0147235
dnn_1200 0.0169271
dnn_800 0.0170273
dnn_400 0.0195312
dnn_600_1200 0.0188301
dnn_do_400 0.0127203
dnn_do_800 0.0146234
dnn_do_1200 0.0153245
dnn_do_600_1200 0.0137219


In [51]:
for name, ter in eval_dict.items():
    print(name, round(ter, 7))

bnn_sgp_1200 0.01362
bnn_sgp_800 0.01502
bnn_sgp_400 0.01442
bnn_sgp_600_1200 0.01152
bnn_sgp_600_800 0.01252
bnn_sgp_600_400 0.01382
bnn_mog_600_1200 0.01412
bnn_mog_600_800 0.01442
bnn_mog_600_400 0.01442
bnn_mog_1200 0.01482
bnn_mog_800 0.01562
bnn_mog_400 0.01472
dnn_1200 0.01663
dnn_800 0.01683
dnn_400 0.02043
dnn_600_1200 0.01893
dnn_do_400 0.01332
dnn_do_800 0.01412
dnn_do_1200 0.01522
dnn_do_600_1200 0.01372
